In [70]:
import pickle
import statistics
import numpy as np
import pandas as pd
from datetime import datetime

Functions # todo: import from other notebook

In [71]:
def sin_transform(values):
    return np.sin(2 * np.pi * values / len(set(values)))

def cos_transform(values):
    return np.cos(2 * np.pi * values / len(set(values)))

In [72]:
def date_engineering(data): # data could be any dataframe that needs date engineering

    data['date'] = pd.to_datetime(data.date, format='%m/%d/%y %H:%M:%S')
    data = data.astype({"date": str})
    #Extract features from date
    data["year"] = data["date"].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').year)
    data["month"] = data["date"].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').month)
    data["weekday"] = data["date"].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').weekday())
    data["week"] = data["date"].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').isocalendar()[1])
    data["day"] = data["date"].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').day)

    # Sin transformation in date features
    data["month_sin"] = sin_transform(data["month"])
    data["weekday_sin"] = sin_transform(data["weekday"])
    data["week_sin"] = sin_transform(data["week"])
    data["day_sin"] = sin_transform(data["day"])

    # Cosine transformation in date features
    data["month_cos"] = cos_transform(data["month"])
    data["weekday_cos"] = cos_transform(data["weekday"])
    data["week_cos"] = cos_transform(data["week"])
    data["day_cos"] = cos_transform(data["day"])

    data=data.drop(columns=['id', 'date','year', 'month', 'weekday', 'week', 'day'])

    return data


In [73]:
def data_to_baseline_model(df): # df is sema or surveys dataframe

    data=fitbit.merge(df,on=['date','id'])
    data = data.drop_duplicates()
    data = date_engineering(data)

    return data

In [74]:
def weekly_fitbit_frequency(survey): # survey is stai or panas dataframe

    fitbit_survey = pd.DataFrame()
    for user in users:
        user_survey = survey.loc[survey['id'] == user]
        user_fitbit = fitbit.loc[fitbit['id'] == user]
        fitbit_survey = pd.concat([fitbit_survey, user_survey], ignore_index=True)
        for day in list(user_survey['date']):
            weekly_fitbit = user_fitbit.loc[user_fitbit['date'] < day]
            weekly_fitbit = weekly_fitbit.set_index(['date'])
            weekly_fitbit = weekly_fitbit.last('7D')
            weekly_fitbit = weekly_fitbit.drop(columns=['id'])
            for column in weekly_fitbit.columns:
                fitbit_survey[column] = statistics.median(list(weekly_fitbit[column]))
    fitbit_survey = fitbit_survey.drop_duplicates()
    fitbit_survey = date_engineering(fitbit_survey)

    return fitbit_survey

Load Fitbit

In [75]:
fitbit = pd.read_pickle('./data/fitbit_all_numerical.pkl')
fitbit["date"] = pd.to_datetime(pd.to_datetime(fitbit["date"]).dt.date)
fitbit

,id,date,nightly_temperature,nremhr,rmssd,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,responsiveness_points_percentage,...,Martial Arts,Run,Spinning,Sport,Swim,Treadmill,Walk,Weights,Workout,Yoga/Pilates
0,621e2e8e67b776a24055b564,2021-05-24,34.137687,57.432,89.603,14.8,78.0,0.833333,0.675,0.866667,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,621e301e67b776a240608a72,2021-05-24,34.710232,NaN,NaN,NaN,74.0,0.700000,0.700,0.833333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,621e301367b776a24057738e,2021-05-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,621e360b67b776a24039709f,2021-05-24,NaN,68.321,31.413,15.4,69.0,0.633333,0.650,0.800000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,621e2fce67b776a240279baa,2021-05-24,34.783747,83.170,15.941,15.2,70.0,0.733333,0.725,0.633333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4959,621e345267b776a240691064,2022-01-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4960,621e310d67b776a24003096d,2022-01-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4961,621e324e67b776a2400191cb,2022-01-17,33.104768,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4962,621e30f467b776a240f22944,2022-01-17,34.352490,NaN,NaN,13.4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Load TTM

In [76]:
ttm = pd.read_pickle('./data/surveys/ttm.pkl')
ttm = ttm.rename(columns={"user_id": "id", "submitdate": "date"})
ttm["date"] = pd.to_datetime(pd.to_datetime(ttm["date"]).dt.date)
ttm = ttm.drop_duplicates()
ttm = ttm.drop(columns=['type'])
ttm

,id,date,stage,ttm_consciousness_raising,ttm_dramatic_relief,ttm_environmental_reevaluation,ttm_self_reevaluation,ttm_social_liberation,ttm_counterconditioning,ttm_helping_relationships,ttm_reinforcement_management,ttm_self_liberation,ttm_stimulus_control
0,621e2e8e67b776a24055b564,2021-07-26,Maintenance,1.333333,2.666667,2.333333,4.333333,4.000000,4.000000,4.000000,4.000000,4.000000,1.666667
0,621e2eaf67b776a2406b14ac,2021-11-29,Contemplation,3.333333,2.666667,4.000000,4.666667,4.000000,2.333333,3.000000,4.333333,3.333333,1.000000
1,621e2eaf67b776a2406b14ac,2022-01-17,Action,3.666667,2.333333,4.333333,4.333333,3.333333,1.666667,3.000000,4.333333,3.333333,3.000000
0,621e2ed667b776a24085d8d1,2021-06-07,Preparation,1.000000,2.000000,3.000000,3.333333,3.333333,1.000000,1.000000,3.333333,1.666667,1.000000
1,621e2ed667b776a24085d8d1,2021-07-30,Contemplation,1.666667,2.333333,4.000000,3.333333,3.666667,1.333333,1.000000,2.333333,2.333333,1.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,621e36c267b776a240ba2756,2021-05-31,Action,2.666667,1.000000,3.000000,4.333333,3.666667,4.000000,3.333333,4.000000,3.666667,3.000000
0,621e36f967b776a240e5e7c9,2021-05-31,Action,2.000000,4.333333,3.333333,5.000000,3.000000,3.333333,1.666667,3.666667,4.000000,3.333333
1,621e36f967b776a240e5e7c9,2021-07-26,Action,4.000000,3.666667,3.666667,5.000000,3.000000,3.000000,1.666667,4.666667,4.000000,3.000000
0,621e375b67b776a240290cdc,2021-06-01,Action,2.333333,1.666667,1.000000,2.666667,5.000000,1.000000,3.000000,4.666667,2.666667,1.333333


Merge Fitbit with TTM

In [77]:
data = data_to_baseline_model(ttm)
data

,nightly_temperature,nremhr,rmssd,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,responsiveness_points_percentage,daily_temperature_variation,calories,...,ttm_self_liberation,ttm_stimulus_control,month_sin,weekday_sin,week_sin,day_sin,month_cos,weekday_cos,week_cos,day_cos
0,33.124766,75.366,22.920,16.6,75.0,0.933333,0.800,0.500000,-0.819009,2034.95,...,2.333333,2.000000,-0.866025,0.0,-1.000000,-0.735724,0.5,1.0,-2.449913e-15,-0.677282
1,34.379283,73.504,21.036,15.2,82.0,0.966667,0.825,0.666667,-1.125404,2605.75,...,2.666667,3.000000,-0.866025,0.0,-1.000000,-0.735724,0.5,1.0,-2.449913e-15,-0.677282
2,34.312944,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.812942,3242.22,...,4.000000,2.333333,-0.866025,0.0,-1.000000,-0.735724,0.5,1.0,-2.449913e-15,-0.677282
3,33.658822,63.698,33.857,14.8,88.0,0.766667,1.000,0.833333,-1.383988,2880.37,...,4.333333,3.000000,-0.866025,0.0,-1.000000,-0.735724,0.5,1.0,-2.449913e-15,-0.677282
4,34.847500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.840776,2024.81,...,3.333333,2.333333,-0.866025,0.0,-1.000000,-0.735724,0.5,1.0,-2.449913e-15,-0.677282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,NaN,57.327,73.746,16.8,NaN,NaN,NaN,NaN,NaN,2356.75,...,5.000000,4.000000,0.866025,0.0,0.707107,-0.614213,0.5,1.0,-7.071068e-01,0.789141
72,34.647437,60.891,54.032,12.0,85.0,0.700000,0.925,0.900000,-3.002026,2660.51,...,3.000000,1.000000,0.866025,0.0,0.707107,-0.614213,0.5,1.0,-7.071068e-01,0.789141
73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1120.51,...,1.333333,1.000000,0.866025,0.0,0.707107,-0.614213,0.5,1.0,-7.071068e-01,0.789141
74,34.237581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.165995,1602.67,...,3.333333,2.666667,0.866025,0.0,0.707107,-0.614213,0.5,1.0,-7.071068e-01,0.789141


In [78]:
data.to_pickle('./data/fitbit_ttm_to_model.pkl')

Load BREQ

In [79]:
breq = pd.read_pickle('./data/surveys/breq.pkl')
breq = breq.rename(columns={"user_id": "id", "submitdate": "date"})
breq["date"] = pd.to_datetime(pd.to_datetime(breq["date"]).dt.date)
breq = breq.drop_duplicates()
breq = breq.drop(columns=['type'])
breq

,id,date,breq_amotivation,breq_external_regulation,breq_introjected_regulation,breq_identified_regulation,breq_intrinsic_regulation,breq_self_determination
0,621e2e8e67b776a24055b564,2021-05-31,1.00,1.00,1.000000,3.50,4.25,intrinsic_regulation
1,621e2e8e67b776a24055b564,2021-07-26,1.00,1.00,1.666667,3.50,4.00,intrinsic_regulation
0,621e2eaf67b776a2406b14ac,2021-11-29,1.25,1.00,1.666667,4.50,3.75,identified_regulation
1,621e2eaf67b776a2406b14ac,2022-01-17,1.00,1.25,2.333333,4.00,3.50,identified_regulation
0,621e2ed667b776a24085d8d1,2021-06-07,2.75,2.00,3.000000,2.50,1.00,introjected_regulation
...,...,...,...,...,...,...,...,...
0,621e36c267b776a240ba2756,2021-05-31,1.00,1.00,1.000000,3.75,4.75,intrinsic_regulation
0,621e36f967b776a240e5e7c9,2021-05-31,1.00,1.75,4.666667,4.00,3.25,introjected_regulation
1,621e36f967b776a240e5e7c9,2021-07-26,1.00,1.75,5.000000,4.75,3.25,introjected_regulation
0,621e375b67b776a240290cdc,2021-06-01,1.00,1.00,1.666667,3.50,2.75,identified_regulation


Merge Fitbit with BREQ

In [80]:
data = data_to_baseline_model(breq)
data

,nightly_temperature,nremhr,rmssd,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,responsiveness_points_percentage,daily_temperature_variation,calories,...,breq_intrinsic_regulation,breq_self_determination,month_sin,weekday_sin,week_sin,day_sin,month_cos,weekday_cos,week_cos,day_cos
0,33.124766,75.366,22.920,16.6,75.0,0.933333,0.800,0.500000,-0.819009,2034.95,...,3.00,intrinsic_regulation,-0.866025,0.0,-1.000000,-0.984808,0.5,1.0,-2.449913e-15,-0.173648
1,34.379283,73.504,21.036,15.2,82.0,0.966667,0.825,0.666667,-1.125404,2605.75,...,4.50,intrinsic_regulation,-0.866025,0.0,-1.000000,-0.984808,0.5,1.0,-2.449913e-15,-0.173648
2,34.312944,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.812942,3242.22,...,3.25,identified_regulation,-0.866025,0.0,-1.000000,-0.984808,0.5,1.0,-2.449913e-15,-0.173648
3,33.658822,63.698,33.857,14.8,88.0,0.766667,1.000,0.833333,-1.383988,2880.37,...,4.50,intrinsic_regulation,-0.866025,0.0,-1.000000,-0.984808,0.5,1.0,-2.449913e-15,-0.173648
4,34.847500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.840776,2024.81,...,4.25,intrinsic_regulation,-0.866025,0.0,-1.000000,-0.984808,0.5,1.0,-2.449913e-15,-0.173648
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,NaN,57.327,73.746,16.8,NaN,NaN,NaN,NaN,NaN,2356.75,...,5.00,intrinsic_regulation,0.866025,0.0,0.707107,-0.342020,0.5,1.0,-7.071068e-01,0.939693
72,34.647437,60.891,54.032,12.0,85.0,0.700000,0.925,0.900000,-3.002026,2660.51,...,2.00,identified_regulation,0.866025,0.0,0.707107,-0.342020,0.5,1.0,-7.071068e-01,0.939693
73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1120.51,...,4.00,intrinsic_regulation,0.866025,0.0,0.707107,-0.342020,0.5,1.0,-7.071068e-01,0.939693
74,34.237581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.165995,1602.67,...,3.50,intrinsic_regulation,0.866025,0.0,0.707107,-0.342020,0.5,1.0,-7.071068e-01,0.939693


In [81]:
data.to_pickle('./data/fitbit_breq_to_model.pkl')

Load Personality

In [82]:
big5 = pd.read_pickle('./data/surveys/personality.pkl')
big5 = big5.rename(columns={"user_id": "id", "submitdate": "date"})
big5["date"] = pd.to_datetime(pd.to_datetime(big5["date"]).dt.date)
big5 = big5.drop_duplicates()
big5 = big5.drop(columns=['type'])
big5

,id,date,extraversion,agreeableness,conscientiousness,stability,intellect,gender,ipip_extraversion_category,ipip_agreeableness_category,ipip_conscientiousness_category,ipip_stability_category,ipip_intellect_category
0,621e2e8e67b776a24055b564,2021-05-31,21.0,33.0,45.0,42.0,40.0,MALE,LOW,LOW,HIGH,HIGH,AVERAGE
1,621e2eaf67b776a2406b14ac,2021-11-29,32.0,45.0,30.0,18.0,41.0,FEMALE,AVERAGE,HIGH,AVERAGE,LOW,HIGH
2,621e2ed667b776a24085d8d1,2021-06-07,40.0,43.0,22.0,28.0,34.0,FEMALE,HIGH,AVERAGE,LOW,AVERAGE,AVERAGE
3,621e2f3967b776a240c654db,2021-05-31,25.0,34.0,30.0,39.0,37.0,MALE,AVERAGE,AVERAGE,LOW,HIGH,AVERAGE
4,621e2f6167b776a240e082a9,2021-06-01,41.0,41.0,30.0,34.0,30.0,FEMALE,HIGH,AVERAGE,AVERAGE,HIGH,LOW
5,621e2f7a67b776a240f14425,2021-05-31,29.0,38.0,43.0,39.0,35.0,MALE,AVERAGE,AVERAGE,HIGH,HIGH,AVERAGE
6,621e2f9167b776a240011ccb,2021-11-28,21.0,45.0,31.0,18.0,39.0,FEMALE,LOW,HIGH,AVERAGE,LOW,HIGH
7,621e2fb367b776a24015accd,2021-05-31,37.0,36.0,29.0,27.0,41.0,MALE,HIGH,AVERAGE,LOW,LOW,HIGH
8,621e2fce67b776a240279baa,2021-05-31,34.0,42.0,35.0,27.0,35.0,MALE,HIGH,HIGH,AVERAGE,LOW,AVERAGE
9,621e2ff067b776a2403eb737,2021-11-29,33.0,44.0,23.0,11.0,30.0,FEMALE,AVERAGE,HIGH,LOW,LOW,LOW


Merge Fitbit with Personality

In [83]:
data = data_to_baseline_model(big5)
data

,nightly_temperature,nremhr,rmssd,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,responsiveness_points_percentage,daily_temperature_variation,calories,...,ipip_stability_category,ipip_intellect_category,month_sin,weekday_sin,week_sin,day_sin,month_cos,weekday_cos,week_cos,day_cos
0,33.124766,75.366,22.920,16.6,75.0,0.933333,0.800,0.500000,-0.819009,2034.95,...,HIGH,AVERAGE,-2.449294e-16,0.000000,-8.660254e-01,-3.826834e-01,1.000000,1.000000,-0.5,9.238795e-01
1,34.379283,73.504,21.036,15.2,82.0,0.966667,0.825,0.666667,-1.125404,2605.75,...,LOW,AVERAGE,-2.449294e-16,0.000000,-8.660254e-01,-3.826834e-01,1.000000,1.000000,-0.5,9.238795e-01
2,34.312944,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.812942,3242.22,...,LOW,HIGH,-2.449294e-16,0.000000,-8.660254e-01,-3.826834e-01,1.000000,1.000000,-0.5,9.238795e-01
3,33.658822,63.698,33.857,14.8,88.0,0.766667,1.000,0.833333,-1.383988,2880.37,...,LOW,LOW,-2.449294e-16,0.000000,-8.660254e-01,-3.826834e-01,1.000000,1.000000,-0.5,9.238795e-01
4,34.847500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.840776,2024.81,...,HIGH,HIGH,-2.449294e-16,0.000000,-8.660254e-01,-3.826834e-01,1.000000,1.000000,-0.5,9.238795e-01
5,34.159081,NaN,NaN,NaN,0.0,0.000000,0.000,0.000000,-2.169940,3947.92,...,LOW,AVERAGE,-2.449294e-16,0.000000,-8.660254e-01,-3.826834e-01,1.000000,1.000000,-0.5,9.238795e-01
6,34.476865,58.652,109.509,14.8,84.0,0.933333,0.725,0.900000,-2.577960,2300.02,...,HIGH,AVERAGE,-2.449294e-16,0.000000,-8.660254e-01,-3.826834e-01,1.000000,1.000000,-0.5,9.238795e-01
7,33.562092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.463412,2545.38,...,HIGH,AVERAGE,-2.449294e-16,0.000000,-8.660254e-01,-3.826834e-01,1.000000,1.000000,-0.5,9.238795e-01
8,35.560128,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.501502,1724.92,...,AVERAGE,LOW,-2.449294e-16,0.000000,-8.660254e-01,-3.826834e-01,1.000000,1.000000,-0.5,9.238795e-01
9,33.876789,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.522155,2116.55,...,HIGH,AVERAGE,-2.449294e-16,0.000000,-8.660254e-01,-3.826834e-01,1.000000,1.000000,-0.5,9.238795e-01


In [84]:
data.to_pickle('./data/fitbit_personality_to_model.pkl')

Find the users

In [85]:
users = list(fitbit['id'].unique())

Load PANAS

In [86]:
panas = pd.read_pickle('./data/surveys/panas.pkl')
panas = panas.rename(columns={"user_id": "id", "submitdate": "date"})
panas["date"] = pd.to_datetime(pd.to_datetime(panas["date"]).dt.date)
panas = panas.drop_duplicates()
panas = panas.drop(columns=['type'])
panas

,id,date,positive_affect_score,negative_affect_score
1,621e2e8e67b776a24055b564,2021-07-26,37,14
2,621e2e8e67b776a24055b564,2021-05-31,38,12
3,621e2e8e67b776a24055b564,2021-06-07,37,12
4,621e2e8e67b776a24055b564,2021-06-28,31,12
5,621e2e8e67b776a24055b564,2021-07-19,37,12
...,...,...,...,...
1,621e375367b776a24021e950,2022-01-02,30,33
0,621e375b67b776a240290cdc,2021-06-01,27,29
1,621e375b67b776a240290cdc,2021-06-08,30,30
2,621e375b67b776a240290cdc,2021-06-22,30,30


In [87]:
fitbit_panas = weekly_fitbit_frequency(panas)
fitbit_panas

,positive_affect_score,negative_affect_score,nightly_temperature,nremhr,rmssd,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,responsiveness_points_percentage,...,Workout,Yoga/Pilates,month_sin,weekday_sin,week_sin,day_sin,month_cos,weekday_cos,week_cos,day_cos
0,37,14,34.674633,51.192,49.232,12.2,83.0,0.733333,0.925,0.8,...,0.0,0.0,-7.071068e-01,0.000000,7.557496e-01,-0.743145,7.071068e-01,1.000000,-0.654861,0.669131
1,38,12,34.674633,51.192,49.232,12.2,83.0,0.733333,0.925,0.8,...,0.0,0.0,-7.071068e-01,0.000000,-1.133108e-15,0.207912,-7.071068e-01,1.000000,1.000000,0.978148
2,37,12,34.674633,51.192,49.232,12.2,83.0,0.733333,0.925,0.8,...,0.0,0.0,-1.000000e+00,0.000000,2.817326e-01,0.994522,-1.836970e-16,1.000000,0.959493,0.104528
3,31,12,34.674633,51.192,49.232,12.2,83.0,0.733333,0.925,0.8,...,0.0,0.0,-1.000000e+00,0.000000,9.096320e-01,-0.406737,-1.836970e-16,1.000000,0.415415,0.913545
4,37,12,34.674633,51.192,49.232,12.2,83.0,0.733333,0.925,0.8,...,0.0,0.0,-7.071068e-01,0.000000,9.096320e-01,-0.743145,7.071068e-01,1.000000,-0.415415,-0.669131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,44,13,34.674633,51.192,49.232,12.2,83.0,0.733333,0.925,0.8,...,0.0,0.0,3.673940e-16,0.000000,9.096320e-01,-0.866025,-1.000000e+00,1.000000,-0.415415,-0.500000
264,45,11,34.674633,51.192,49.232,12.2,83.0,0.733333,0.925,0.8,...,0.0,0.0,3.673940e-16,0.000000,7.557496e-01,-0.587785,-1.000000e+00,1.000000,-0.654861,0.809017
265,44,11,34.674633,51.192,49.232,12.2,83.0,0.733333,0.925,0.8,...,0.0,0.0,7.071068e-01,0.000000,2.817326e-01,0.587785,7.071068e-01,1.000000,0.959493,0.809017
266,45,13,34.674633,51.192,49.232,12.2,83.0,0.733333,0.925,0.8,...,0.0,0.0,7.071068e-01,0.974928,5.406408e-01,0.587785,7.071068e-01,-0.222521,0.841254,-0.809017


In [88]:
data.to_pickle('./data/fitbit_panas_to_model.pkl')

Load STAI

In [89]:
stai = pd.read_pickle('./data/surveys/stai.pkl')
stai = stai.rename(columns={"user_id": "id", "submitdate": "date"})
stai["date"] = pd.to_datetime(pd.to_datetime(stai["date"]).dt.date)
panas = panas.drop_duplicates()
stai = stai.drop(columns=['type'])
stai

,id,date,stai_stress,stai_stress_category
0,621e2e8e67b776a24055b564,2021-05-31,45.0,Below average
1,621e2e8e67b776a24055b564,2021-06-07,46.0,Average
3,621e2e8e67b776a24055b564,2021-07-19,46.0,Average
4,621e2e8e67b776a24055b564,2021-07-26,46.0,Average
5,621e2e8e67b776a24055b564,2021-06-28,54.0,Above average
...,...,...,...,...
0,621e375b67b776a240290cdc,2021-06-08,60.0,Above average
1,621e375b67b776a240290cdc,2021-07-06,60.0,Above average
2,621e375b67b776a240290cdc,2021-06-01,59.0,Above average
3,621e375b67b776a240290cdc,2021-06-22,60.0,Above average


In [90]:
fitbit_stai = weekly_fitbit_frequency(stai)
fitbit_stai

,stai_stress,stai_stress_category,nightly_temperature,nremhr,rmssd,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,responsiveness_points_percentage,...,Workout,Yoga/Pilates,month_sin,weekday_sin,week_sin,day_sin,month_cos,weekday_cos,week_cos,day_cos
0,45.0,Below average,34.073346,51.364,51.324,12.2,84.0,0.833333,0.875,0.866667,...,0.0,0.0,-7.071068e-01,0.000000,-1.133108e-15,0.207912,-7.071068e-01,1.000000,1.000000,0.978148
1,46.0,Average,34.073346,51.364,51.324,12.2,84.0,0.833333,0.875,0.866667,...,0.0,0.0,-1.000000e+00,0.000000,2.817326e-01,0.994522,-1.836970e-16,1.000000,0.959493,0.104528
2,46.0,Average,34.073346,51.364,51.324,12.2,84.0,0.833333,0.875,0.866667,...,0.0,0.0,-7.071068e-01,0.000000,9.096320e-01,-0.743145,7.071068e-01,1.000000,-0.415415,-0.669131
3,46.0,Average,34.073346,51.364,51.324,12.2,84.0,0.833333,0.875,0.866667,...,0.0,0.0,-7.071068e-01,0.000000,7.557496e-01,-0.743145,7.071068e-01,1.000000,-0.654861,0.669131
4,54.0,Above average,34.073346,51.364,51.324,12.2,84.0,0.833333,0.875,0.866667,...,0.0,0.0,-1.000000e+00,0.000000,9.096320e-01,-0.406737,-1.836970e-16,1.000000,0.415415,0.913545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,49.0,Average,34.073346,51.364,51.324,12.2,84.0,0.833333,0.875,0.866667,...,0.0,0.0,7.071068e-01,0.000000,7.557496e-01,-0.994522,-7.071068e-01,1.000000,0.654861,-0.104528
275,51.0,Above average,34.073346,51.364,51.324,12.2,84.0,0.833333,0.875,0.866667,...,0.0,0.0,3.673940e-16,0.000000,9.898214e-01,0.951057,-1.000000e+00,1.000000,0.142315,0.309017
276,45.0,Below average,34.073346,51.364,51.324,12.2,84.0,0.833333,0.875,0.866667,...,0.0,0.0,3.673940e-16,0.781831,9.898214e-01,0.207912,-1.000000e+00,0.623490,-0.142315,-0.978148
277,48.0,Average,34.073346,51.364,51.324,12.2,84.0,0.833333,0.875,0.866667,...,0.0,0.0,3.673940e-16,0.000000,9.096320e-01,-0.866025,-1.000000e+00,1.000000,-0.415415,-0.500000


In [91]:
data.to_pickle('./data/fitbit_stai_to_model.pkl')